In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import joblib  # Importando joblib para salvar o modelo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from torch.utils.data import DataLoader, TensorDataset
from AlzheimerCNN import AlzheimerNet

# 1️⃣ Carregar e pré-processar os dados
df = pd.read_csv("D:\\Projetos\\AlzheimerEarlyDetection_v4\\alzheimers_disease_data.csv")

# Mapear Diagnosis para nova classificação
def map_diagnosis(row):
    if row['Diagnosis'] == 0:
        return 0  # Sem Demência
    elif row['MMSE'] >= 24:
        return 1  # Demência Leve
    elif row['MMSE'] >= 18:
        return 2  # Demência Moderada
    else:
        return 3  # Demência Crônica

df['Diagnosis_New'] = df.apply(map_diagnosis, axis=1)

# Separar features e target
features = df.drop(columns=['PatientID', 'Diagnosis', 'Diagnosis_New', 'DoctorInCharge'])
target = df['Diagnosis_New']

# Codificar variáveis categóricas
for col in features.select_dtypes(include=['object']).columns:
    features[col] = LabelEncoder().fit_transform(features[col])

# Normalizar os dados
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Converter para tensores do PyTorch
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Criar o modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 2️⃣ Treinar o modelo
model.fit(X_train, y_train)

# 3️⃣ Avaliação
accuracy = model.score(X_test, y_test)
print(f"Acurácia do modelo: {accuracy:.2f}%")

# 4️⃣ Salvar o modelo com joblib
joblib.dump(model, 'alzheimer_csv_model.joblib')  # Salvar o modelo com joblib

# Salvar o scaler (caso seja necessário para usar os dados novos posteriormente)
joblib.dump(scaler, 'scaler.pkl')


Acurácia do modelo: 0.88%


['scaler.pkl']